# Система непересекающихся множеств

$\def\Un{\operatorname{Union}}$
$\def\Find{\operatorname{Find}}$
$\def\MakeSet{\operatorname{MakeSet}}$

![Иллюстрация к проекту](https://habrastorage.org/storage/6cf7f9c4/e6848578/0bf0505b/5d3ded44.png)

Система непересекающихся множеств - иерархическая структура данных, позволяющая эффективно работать с множествами.

В самом начале есть лишь разбросанные элементы, которые по сути являются множествами из одного элемента. Будут введены две операции: $\operatorname{Union(x, y)}, \Find(x), \MakeSet(x)$

$\operatorname{Union}(x, y)$ - будет объединять два множества в одно, а $\Find(x)$ - говорить в каком множестве лежит $х$. Точнее, мы будем выбирать какого-то представителя множества и на $\Find$ выдавать именно его. Представителем множества может являться любой элемент множества.
$\MakeSet(x)$ - создаёт множество из одного элемента $x$.
    
Рассмотрим несколько вариантов реализации данной структуры данных.

Первый вариант. Создадим массив $\mathcal{P}$, где $\mathcal{P}_i$ - представитель множества, в котором содержится элемент $i$. Нумерация с 1.


In [1]:
class DisjointSetForest_Naive1:
    
    def __init__(self): 
        self.p = list()
        
    def MakeSet(self, x): 
        self.p.append(x)
        
    def Find(self, x):
        if x < len(self.p):
            return self.p[x-1]
    
    def Print(self):
        print(self.p)
    
    def Union(self, x, y):
        x, y = self.Find(x), self.Find(y)
        if x == y:
            pass
        for i in range(len(self.p)):
            if self.p[i] == x:
                self.p[i] = y

In [2]:
dsf = DisjointSetForest_Naive1()

In [3]:
dsf.MakeSet(1)

In [4]:
dsf.Print()

[1]


In [5]:
dsf.MakeSet(2)

In [6]:
dsf.MakeSet(3)
dsf.MakeSet(4)
dsf.MakeSet(5)
dsf.MakeSet(6)
dsf.MakeSet(7)

In [7]:
dsf.Print()

[1, 2, 3, 4, 5, 6, 7]


In [8]:
dsf.Find(3)

3

In [9]:
dsf.Union(1, 3)

In [10]:
dsf.Print()

[3, 2, 3, 4, 5, 6, 7]


In [11]:
dsf.Union(4, 5)

In [13]:
dsf.Union(6, 5)

In [14]:
dsf.Print()

[3, 2, 3, 5, 5, 5, 7]


Также можно хранить "списки подчиненных" для каждой вершины. И использовать хак с подвязкой по размеру. Выгоднее будет подвязывать маленькие множества к большим. Так как пробегаться в итоге нужно будет по меньшему количеству.

Мы же рассмотрим вариант представления в виде деревьев.
Пусть каждая вершина будет знать своего родителя. А родителем является представитель множества. 
Обозначим - $\operatorname{Rep}(x)$ - представитель множества

$\mathcal{P}$ - массив, такой что 
$\mathcal{P}_i = \begin{cases}
                    i & i = \operatorname{Rep}(i) \\
                    \operatorname{Rep}(i) & \text{иначе}
                 \end{cases}$


In [35]:
class DisjointSetForest_Tree:
    
    def __init__(self):
        self.p = list()
        self.p.append(-1)
        
    def MakeSet(self, x):
        self.p.append(x)
        
    def Find(self, x):
        while self.p[x] != x:
            x = self.p[x]
        return x

    def Union(self, x, y):
        x, y = self.Find(x), self.Find(y)
        self.p[x] = y
        
    def Print(self):
        print(self.p[1:])

In [36]:
dsj = DisjointSetForest_Tree()

In [37]:
dsj.MakeSet(1)
dsj.MakeSet(2)
dsj.MakeSet(3)
dsj.MakeSet(4)
dsj.MakeSet(5)

In [38]:
dsj.Print()

[1, 2, 3, 4, 5]


In [39]:
dsj.Union(1, 2)

In [40]:
dsj.Print()

[2, 2, 3, 4, 5]


In [41]:
dsj.Union(3,2)

In [42]:
dsj.Print()

[2, 2, 2, 4, 5]


In [43]:
dsj.Find(1)

2

In [44]:
dsj.Union(4, 3)

In [45]:
dsj.Print()

[2, 2, 2, 2, 5]


In [46]:
dsj.Union(1, 5)

In [47]:
dsj.Print()

[2, 5, 2, 2, 5]


In [48]:
dsj.Union(0, 5)

In [49]:
dsj.Print()

[2, 5, 2, 2, 5]


В целом, такой вариант тоже работает. Для простоты был введен фиктивный элемент на 0 позиции. Можно достичь ускорения за счет того, что мы будем подвязывать меньшие деревья к большим. Сделаем полную реализацию вместе с хранением каких-нибудь данных, пусть, это будут, допустим строки.

In [328]:
class DisjointSetForest_Tree_Final:
    
    def __init__(self):
        self.p = list()
        self.length = 0
        self.par_sizes = []
        
    def MakeSet(self, value):
        self.p.append([self.length, value])
        self.length += 1 
        self.par_sizes.append(1)
        
    def __GetIdx(self, value):
        idx = -1
        for par, val in self.p:
            if val == value:
                idx = par
                break
        if idx == -1:
            raise Exception("Not finded element in set")
        return idx
        
    def Find(self, x):
        idx = self.__GetIdx(x)
        """while self.p[idx][0] != idx:
            idx = self.p[idx][0]"""
        # улучшение:
        while self.p[idx][0] != idx:
            self.p[idx][0] = self.Find(self.p[idx][1])
        return self.p[idx][0]
    
    def Union(self, value1, value2):
        x, y = self.Find(value1), self.Find(value2)
            
        self.p[x][0] = y
        self.par_sizes[y] += 1
    
    def Print(self):
        i = 0
        for parent, value in self.p:
            print("<{0}> [Parent: {1}, value: {2}]".format(i, parent, value))
            i += 1
        print(self.par_sizes)
            

In [329]:
dsjf = DisjointSetForest_Tree_Final()

In [330]:
dsjf.MakeSet("Иван")

In [331]:
dsjf.Print()

<0> [Parent: 0, value: Иван]
[1]


In [332]:
dsjf.MakeSet("Костя")
dsjf.MakeSet("Кирилл")
dsjf.MakeSet("Сергей")
dsjf.MakeSet("Наташа")

In [333]:
dsjf.Print()

<0> [Parent: 0, value: Иван]
<1> [Parent: 1, value: Костя]
<2> [Parent: 2, value: Кирилл]
<3> [Parent: 3, value: Сергей]
<4> [Parent: 4, value: Наташа]
[1, 1, 1, 1, 1]


In [334]:
dsjf.Find("Костя")

1

In [335]:
dsjf.Union("Костя", "Наташа")

In [336]:
dsjf.Print()

<0> [Parent: 0, value: Иван]
<1> [Parent: 4, value: Костя]
<2> [Parent: 2, value: Кирилл]
<3> [Parent: 3, value: Сергей]
<4> [Parent: 4, value: Наташа]
[1, 1, 1, 1, 2]


In [337]:
dsjf.Union("Костя", "Сергей")

In [338]:
dsjf.Print()

<0> [Parent: 0, value: Иван]
<1> [Parent: 4, value: Костя]
<2> [Parent: 2, value: Кирилл]
<3> [Parent: 3, value: Сергей]
<4> [Parent: 3, value: Наташа]
[1, 1, 1, 2, 2]


In [339]:
dsjf.Union("Кирилл", "Сергей")

In [340]:
dsjf.Print()

<0> [Parent: 0, value: Иван]
<1> [Parent: 4, value: Костя]
<2> [Parent: 3, value: Кирилл]
<3> [Parent: 3, value: Сергей]
<4> [Parent: 3, value: Наташа]
[1, 1, 1, 3, 2]


$\operatorname{\_\_GetIdx(x)}$ работает за $O(n)$ в худшем случае. (Это из-за хранения значений на вершинах).

Также нет смысла каждый раз искать родителя. Можно в $\Find(x)$ сразу записывать нужного родителя. 

$\Find(x)$ - если дерево получается вырожденным, то в худшем случае - $O(n)$.
$\Un(x)$ работает также. Но можно сделать одно простое улучшение. Тогда дерево не станет вырожденным и будет ускорение операций. Необходимо подвешивать меньшее дерево к большему (можно использовать глубину, можно количество вершин)

Без хранения значений на вершинах можно получить хорошую скорость порядка итерационного логарифма. А на самом деле <b>Функции Аккермана</b>